In [1]:
import math
import os
import sys
import time
from queue import PriorityQueue

In [2]:
class Tablero:
    def __init__(self, estados):
        self.tamano = int(math.sqrt(len(estados)))
        self.estados = estados

    def ejecutar_accion(self, accion):
        nuevos_estados = self.estados[:]
        indice_vacio = nuevos_estados.index('0')
        if accion == 'L':
            if indice_vacio % self.tamano > 0:
                nuevos_estados[indice_vacio - 1], nuevos_estados[indice_vacio] = nuevos_estados[indice_vacio], nuevos_estados[indice_vacio - 1]
        if accion == 'R':
            if indice_vacio % self.tamano < (self.tamano - 1):
                nuevos_estados[indice_vacio + 1], nuevos_estados[indice_vacio] = nuevos_estados[indice_vacio], nuevos_estados[indice_vacio + 1]
        if accion == 'U':
            if indice_vacio - self.tamano >= 0:
                nuevos_estados[indice_vacio - self.tamano], nuevos_estados[indice_vacio] = nuevos_estados[indice_vacio], nuevos_estados[
                    indice_vacio - self.tamano]
        if accion == 'D':
            if indice_vacio + self.tamano < self.tamano * self.tamano:
                nuevos_estados[indice_vacio + self.tamano], nuevos_estados[indice_vacio] = nuevos_estados[indice_vacio], nuevos_estados[
                    indice_vacio + self.tamano]
        return Tablero(nuevos_estados)

In [3]:
class Nodo:
    def __init__(self, estado, padre, accion):
        self.estado = estado
        self.padre = padre
        self.accion = accion
#         self.costo = costo
    def __repr__(self):
        return str(self.estado.estados)

    def __eq__(self, otro):
        return self.estado.estados == otro.estado.estados

    def __hash__(self):
        return hash(self.estado)

In [4]:
def get_hijos(padre_Nodo):
    hijos = []
    accions = ['L', 'R', 'U', 'D']
    for accion in accions:
        hijo_estado = padre_Nodo.estado.ejecutar_accion(accion)
        hijo_Nodo = Nodo(hijo_estado, padre_Nodo, accion)
        hijos.append(hijo_Nodo)
    return hijos

In [5]:
def gcalc(Nodo):
    ''' calcula g(n): encuentra el costo del estado actual a partir del estado origen o inicial'''
    contador = 0
    while Nodo.padre is not None:
        Nodo = Nodo.padre
        contador += 1
    return contador

In [6]:
def hamming(estados):
    ''' heuristicaa Hamming: cuenta el numero de posiciones erroneas en diferentes estados'''
    numero_indices_no_ubicados = 0
    objetivo_estados = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '0']
    for i in objetivo_estados:
        if objetivo_estados.index(i) - estados.index(i) != 0 and i != 0:
            numero_indices_no_ubicados += 1
    return numero_indices_no_ubicados


def manhattan_calculate(estados):
    '''heuristica Manhattan: cuenta el numero de cuadros a partir de una ubicacion en relacion a su posicion final'''
    contador = 0
    for i in range(0, 15):
        indice = estados.index(str(i + 1))  # por que el rango inicia en 0
        contador += (abs((i / 4) - (indice / 4)) + abs((i % 4) - (indice % 4)))  # %4 es la columna y /4 es la fila
    return contador

In [7]:
def find_path(Nodo):
    '''Devuelve la ruta inversa de un nodo origen'''
    path = []
    while (Nodo.padre is not None):
        path.append(Nodo.accion)
        Nodo = Nodo.padre
    path.reverse()
    return path

In [9]:
def goal_test():
    return ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '0']

In [10]:
def astar(estado_inicial, estado_objetivo, heuristica):
    '''A* Search Algorithm'''
    start_time = time.time()
    frontera = list()
    contador = 0
    visitado = dict()
    frontera.append(estado_inicial)
    visitado[estado_inicial.estado] = estado_inicial
    while frontera:
        minim = []
        holder = []
        for x in frontera:
            if heuristica == 0:
                minim.append(hamming(x.estado.estados) + gcalc(x))  # This is the F = h + g
            elif heuristica == 1:
                minim.append(manhattan_calculate(x.estado.estados) + gcalc(x))
            holder.append(x)
        m = min(minim)  # finds minimum F value
        estado_inicial = holder[minim.index(m)]

        if estado_inicial.estado.estados == estado_objetivo:  # solution found!
            end_time = time.time()
            print("\n\nSolucion:")
            print("Movimientos: " + str(' '.join(find_path(estado_inicial))))
            print("Numero de nodos expandidos: " + str(contador))
            print("Tiempo empleado: " + str(round((end_time - start_time), 3)))
            # print("Memory Used: " + str(sys.gettamanoof(visitado) + sys.gettamanoof(frontera)) + " kb")
            break

        frontera.pop(frontera.index(estado_inicial))
        for hijo in get_hijos(estado_inicial):
            contador += 1
            s = hijo.estado
            if s not in visitado or gcalc(hijo) < gcalc(visitado[s]):
                visitado[s] = hijo
                frontera.append(hijo)


In [13]:
def main():
    ei = ['1', '3', '4', '8', '5', '2', '0', '6', '9', '10', '7', '11', '13', '14', '15', '12']
    # ei = ['0', '15', '14', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '1', '2', '3']

    heuristica = input("Elija la heuristica entre 'H' o 'M' (H es Hamming y M ws Manhattan): ")
    if heuristica == 'H':
        heuristica = 0
    elif heuristica == 'M':
        heuristica = 1

    max_depth = 10
    root = Nodo(Tablero(ei), None, None)
    astar(root, goal_test(), heuristica)
    frontera = []
    frontera.append(root)

# Press the green button in the gutter to run the script.
if __name__ == '__main__':
    main()

Elija la heuristica entre 'H' o 'M' (H es Hamming y M ws Manhattan): M


Solucion:
Movimientos: R U L L D R D R D
Numero de nodos expandidos: 36
Tiempo empleado: 0.001
